In [8]:
import pandas as pd
import numpy as np


def forward(V, A, B, pi):
    #We create an empty Alpha array 10 (lines) x 4 (cols)
    alpha = np.zeros((V.shape[0], A.shape[0]))
    alpha[0, :] = pi * B[:, V[0]]

    for t in range(1, V.shape[0]):
        for j in range(A.shape[0]):
            
            alpha[t, j] = alpha[t - 1].dot(A[:, j]) * B[j, V[t]]

    return alpha


def backward(V, A, B):
    beta = np.zeros((V.shape[0], A.shape[0]))

    # setting beta(T) = 1
    beta[V.shape[0] - 1] = np.ones((A.shape[0]))

    # Loop in backward way from T-1 to
    # Due to python indexing the actual loop will be T-2 to 0
    for t in range(V.shape[0] - 2, -1, -1):
        for j in range(A.shape[0]):
            beta[t, j] = (beta[t + 1] * B[:, V[t + 1]]).dot(A[j, :])

    return beta

"""
def baum_welch(V, A, B, pi, n_iter=100):
    M = A.shape[0]
    T = len(V)

    for n in range(n_iter):
        alpha = forward(V, A, B, pi)
        beta = backward(V, A, B)

        xi = np.zeros((M, M, T - 1))
        for t in range(T - 1):
            denominator = np.dot(np.dot(alpha[t, :].T, A) * B[:, V[t + 1]].T, beta[t + 1, :])
            for i in range(M):
                numerator = alpha[t, i] * A[i, :] * B[:, V[t + 1]].T * beta[t + 1, :].T
                xi[i, :, t] = numerator / denominator

        gamma = np.sum(xi, axis=1)
        A = np.sum(xi, 2) / np.sum(gamma, axis=1).reshape((-1, 1))

        # Add additional T'th element in gamma
        gamma = np.hstack((gamma, np.sum(xi[:, :, T - 2], axis=0).reshape((-1, 1))))

        K = B.shape[1]
        denominator = np.sum(gamma, axis=1)
        for l in range(K):
            B[:, l] = np.sum(gamma[:, V == l], axis=1)

        B = np.divide(B, denominator.reshape((-1, 1)))

    return (A, B)

"""


def viterbi(V, A, B, pi):
    global T, M, omega, prev, S, probability
    T = V.shape[0]
    M = A.shape[0]

    omega = np.zeros((T, M))
    omega[0, :] = np.log(pi * B[:, V[0]])

    prev = np.zeros((T - 1, M))

    for t in range(1, T):
        for j in range(M):
            # Same as Forward Probability
            probability = omega[t - 1] + np.log(A[:, j]) + np.log(B[j, V[t]])

            # This is our most probable state given previous state at time t (1)
            prev[t - 1, j] = np.argmax(probability)

            # This is the probability of the most probable state (2)
            omega[t, j] = np.max(probability)

    # Path Array
    S = np.zeros(T)

    # Find the most probable last hidden state
    last_state = np.argmax(omega[T - 1, :])

    S[0] = last_state

    backtrack_index = 1
    for i in range(T - 2, -1, -1):
        S[backtrack_index] = prev[i, int(last_state)]
        last_state = prev[i, int(last_state)]
        backtrack_index += 1

    # Flip the path array since we were backtracking
    S = np.flip(S, axis=0)

    # Convert numeric values to actual hidden states
    result = []
    for s in S:
        if s == 0:
            result.append("PATH1")
        elif s == 1:
            return result.append("PATH2")
        elif s == 2:
            return result.append("PATH3")
        else:
            result.append("PATH4")

    return result


#states = PATH1, PATH2, PATH3, PATH4 = 0, 1, 2, 3

V = np.array([3, 3, 2, 1, 2, 2, 1, 2, 3, 0])


# Transition Probabilities
#a = np.ones((2, 2))
#a = a / np.sum(a, axis=1)
A = np.array(((0.3, 0.2, 0.2, 0.3), (0.2, 0.3, 0.2, 0.3), (0.2, 0.2, 0.3, 0.3), (0.2, 0.2, 0.2, 0.4)))
A = A / np.sum(A)

# Emission Probabilities
#b = np.array(((1, 3, 5), (2, 4, 6)))
#b = b / np.sum(b, axis=1).reshape((-1, 1))
B = np.array(((0.3, 0.3, 0.3, 0.1), (0.3, 0.3, 0.3, 0.1), (0.3, 0.3, 0.3, 0.1), (0.1, 0.1, 0.1, 0.7)))
B = B / np.sum(B)

# Probabilities for the initial distribution
pi = np.array((0.2, 0.2, 0.2, 0.4))


#a, b = baum_welch(V, a, b, initial_distribution, n_iter=100)

print(viterbi(V, A, B, pi))

['PATH4', 'PATH4', 'PATH1', 'PATH1', 'PATH1', 'PATH1', 'PATH1', 'PATH1', 'PATH4', 'PATH1']


In [2]:
omega

array([[ -5.29831737,  -5.29831737,  -5.29831737,  -2.65926004],
       [ -9.34387176,  -9.34387176,  -9.34387176,  -6.70481443],
       [-12.29081387, -12.29081387, -12.29081387, -12.69627898],
       [-17.4713482 , -17.4713482 , -17.4713482 , -18.56996049],
       [-22.65188254, -22.65188254, -22.65188254, -23.75049482],
       [-27.83241687, -27.83241687, -27.83241687, -28.93102916],
       [-33.0129512 , -33.0129512 , -33.0129512 , -34.11156349],
       [-38.19348553, -38.19348553, -38.19348553, -39.29209782],
       [-44.47263215, -44.47263215, -44.47263215, -42.526722  ],
       [-48.11272144, -48.11272144, -48.11272144, -48.51818655]])

In [4]:
prev

array([[3., 3., 3., 3.],
       [3., 3., 3., 3.],
       [0., 1., 2., 0.],
       [0., 1., 2., 0.],
       [0., 1., 2., 0.],
       [0., 1., 2., 0.],
       [0., 1., 2., 0.],
       [0., 1., 2., 0.],
       [3., 3., 3., 3.]])

In [18]:
probability

array([-50.75177877, -50.75177877, -50.75177877, -48.51818655])

In [19]:
omega[0, :]

array([-5.29831737, -5.29831737, -5.29831737, -2.65926004])

In [3]:
S

array([3., 3., 0., 0., 0., 0., 0., 0., 3., 0.])

In [7]:
M

4